In [1]:
# Impute necessary packages
import numpy as np
import pandas as pd
from fancyimpute import BiScaler
from soft_impute import SoftImpute
from functionsCF import GenerateTrainingSet

#### Create the incomplete matrices for training and testing

In [9]:
# Read jester data from files- point to where data is stored, small set of jester dataset
# 100836 (rows), userId	jokeId	rating	timestamp (columns)
# read in values only
# name the variable "rating"
# YOUR CODE HERE
#raise NotImplementedError()
rating = pd.read_csv('MusicRatings.csv', sep=',').values
print(len(rating))

289409


In [10]:
# Show the top 5 rows
print(rating[:5, :])

[[5.26000000e+02 8.00000000e+01 1.47712125e+00]
 [1.40300000e+03 5.40000000e+01 2.20411998e+00]
 [5.56000000e+02 8.00000000e+01 1.30103000e+00]
 [1.03600000e+03 5.40000000e+01 1.47712125e+00]
 [2.35200000e+03 8.00000000e+01 1.30103000e+00]]


In [11]:
# First, create an empty matrix call it "matrix_incomplete"
# YOUR CODE HERE
#raise NotImplementedError()
matrix_incomplete = np.zeros((len(np.unique(rating[:,0])), len(np.unique(rating[:,1]))))
print(matrix_incomplete.shape)

(2421, 807)


In [13]:
# Second, Since some music may not have any ratings, we only use the music that have ratings. 
# Here we correspondingly change the music IDs to make each column have ratings.
# create an array of all music IDs call it "usedID"
# YOUR CODE HERE
#raise NotImplementedError()
usedID = np.unique(rating[:, 1])
#print(usedID)

# replace the music IDs by their positions in the array we just created
# YOUR CODE HERE
#raise NotImplementedError()
for i in range(len(rating[:,1])):
    rating[:,1][i] = np.where(usedID==rating[:,1][i])[0][0] + 1
print(rating[:,1].size)
print(rating[:,1][289000])

289409
803.0


In [28]:
# Finally, we construct the incomplete matrix, on which the incomplete components are nan by 
# default. 
# Make all components of matrix_incomplete nan by default
# YOUR CODE HERE
#raise NotImplementedError()
matrix_incomplete[:] = np.nan

# create the index pair of the components with ratings. Call it "indices"
# YOUR CODE HERE
#raise NotImplementedError()
indices = np.array(rating[:,0] - 1).astype(int), np.array(rating[:,1] - 1).astype(int)
print(indices)

# change the values in the corresponding positions to the known rating information
# YOUR CODE HERE
#raise NotImplementedError()
matrix_incomplete[indices] = rating[:,2]
print(matrix_incomplete[2420,806])
print(matrix_incomplete[13,1])



(array([ 525, 1402,  555, ...,  383,  229, 1384]), array([ 79,  53,  79, ..., 656, 656, 614]))
nan
1.0


In [29]:
# Obtain the index pairs of the training set and the validation set, with ratio 70%
# Use GenerateTrainingSet() function and name the outputs train_indices and validation_indices
# YOUR CODE HERE
#raise NotImplementedError()
train_indices, validation_indices = GenerateTrainingSet(rating[:,0], rating[:,1], 0.70)


# Now use the index pairs to create the incomplete training test called "matrix_train"
# YOUR CODE HERE
#raise NotImplementedError()
matrix_train = matrix_incomplete.copy()
matrix_train[:] = np.nan
matrix_train[train_indices] = matrix_incomplete[train_indices]

#### Run the softImpute model for collaborative filtering

In [30]:
# Create the BiScaler model call it "biscaler"
# YOUR CODE HERE
#raise NotImplementedError()
biscaler = BiScaler(scale_rows=False, scale_columns=False, max_iters=50, verbose=False)


# Rescale both rows and columns to have zero mean
# Call it matrix_train_normalized
# YOUR CODE HERE
#raise NotImplementedError()
matrix_train_normalized = biscaler.fit_transform(matrix_train)


In [31]:
# Use softImpute to complete the matrix. J means the number of archetypes and rand_seed means the 
# seed for the inner random number generator, verbose control whether outputting algorithm logs.

softImpute = SoftImpute(J = 6, maxit = 200, random_seed = 2033, verbose = False)


In [32]:
# Run the softImpute model on the normalized training set
# Call the output matrix_train_softImpute
# YOUR CODE HERE
#raise NotImplementedError()
matrix_train_softImpute = softImpute.fit(matrix_train_normalized)


# Use the softImpute model to create the predicted matrix. If we set copyto as True, then it 
# directly change the value of matrix_train_normalized
# Call the output matrix_train_filled_normalized
# YOUR CODE HERE
#raise NotImplementedError()
matrix_train_filled_normalized = matrix_train_softImpute.predict(matrix_train_normalized, copyto = False)


# Inverse transformation to undo the scaling
# Call the output matrix_train_filled
# YOUR CODE HERE
#raise NotImplementedError()
matrix_train_filled = biscaler.inverse_transform(matrix_train_filled_normalized)


#### Analysis of the predicted ratings

##### Out-of-sample R^2

In [33]:
# Create the baseline method of the training set using np.average()
# Name the output train_average
# YOUR CODE HERE
#raise NotImplementedError()
train_average = np.average(matrix_train[train_indices])


In [34]:
# Calculate out-of-sample R2 and in-sample R2
# Your results may vary from the lesson due to datasize and training test split.
# Generate validation_mse, training_mse, validation_mse_baseline, and training_mse_baseline
# YOUR CODE HERE
#raise NotImplementedError()
validation_mse = ((matrix_train_filled[validation_indices] - matrix_incomplete[validation_indices]) ** 2).mean()
training_mse = ((matrix_train_filled[train_indices] - matrix_incomplete[train_indices]) ** 2).mean()
validation_mse_baseline = ((train_average - matrix_incomplete[validation_indices]) ** 2).mean()
training_mse_baseline = ((train_average - matrix_incomplete[train_indices]) ** 2).mean()

print("out-of-sample R2: %.4f, in-sample R2: %.4f." % (1 - validation_mse / validation_mse_baseline, 1 - training_mse / training_mse_baseline))

out-of-sample R2: 0.2228, in-sample R2: 0.4991.


In [35]:
# Observe your prediction:
# After matrix completion
print("After matrix completion =", matrix_train_filled[526,80])

After matrix completion = 1.066130728707499
